### TODO
- Try adjusting baudrate to see if wait times (select.select) are affected

In [52]:
#baudrate = 400000.0
baudrate = 115200.0
bits_per_word = (8+0+1)
words_per_update = 3
updates_per_second = baudrate / (words_per_update * bits_per_word)
print 'updates per seconds: ', updates_per_second
print 'update interval: ', 1.0/updates_per_second

updates per seconds:  4266.66666667
update interval:  0.000234375


In [83]:
average_3dof_update_interval = 0.004
dejitter_factor = 10.0

effective_update_interval = average_3dof_update_interval * dejitter_factor

print 'effective update interval (s): ', effective_update_interval
print 'update sample rate: ', 1.0/effective_update_interval
print 'nyquist rate: ', 0.5/effective_update_interval

effective update interval (s):  0.04
update sample rate:  25.0
nyquist rate:  12.5


In [2]:
%load_ext Cython

In [2]:
from Adafruit_BNO055 import BNO055
import time

In [3]:
class BNO055_2(BNO055.BNO055):
    def __init__(self, rst=None, address=BNO055.BNO055_ADDRESS_A, i2c=None, gpio=None,
                 serial_port=None, serial_timeout_sec=5, baudrate=115200, **kwargs):
        # If reset pin is provided save it and a reference to provided GPIO
        # bus (or the default system GPIO bus if none is provided).
        self._rst = rst
        if self._rst is not None:
            if gpio is None:
                import Adafruit_GPIO as GPIO
                gpio = GPIO.get_platform_gpio()
            self._gpio = gpio
            # Setup the reset pin as an output at a high level.
            self._gpio.setup(self._rst, GPIO.OUT)
            self._gpio.set_high(self._rst)
            # Wait a 650 milliseconds in case setting the reset high reset the chip.
            time.sleep(0.65)
        self._serial = None
        self._i2c_device = None
        if serial_port is not None:
            import serial
            # Use serial communication if serial_port name is provided.
            # Open the serial port at 115200 baud, 8N1.  Add a 5 second timeout
            # to prevent hanging if device is disconnected.
            print 'about to create Serial object'
            self._serial = serial.Serial(serial_port, baudrate, timeout=serial_timeout_sec,
                                         writeTimeout=serial_timeout_sec)
        else:
            # Use I2C if no serial port is provided.
            # Assume we're using platform's default I2C bus if none is specified.
            if i2c is None:
                import Adafruit_GPIO.I2C as I2C
                i2c = I2C
            # Save a reference to the I2C device instance for later communication.
            self._i2c_device = i2c.get_i2c_device(address, **kwargs)

In [4]:
def print_measurements(lin_accel,mag,gyro,accel):
    print '({0:.3f}, {1:.3f}, {2:.3f})\t({3:.3f}, {4:.3f}, {5:.3f})\t({6:.3f}, {7:.3f}, {8:.3f})\t({9:.3f}, {10:.3f}, {11:.3f})'.format(
        lin_accel[0],
        lin_accel[1],
        lin_accel[2],
        mag[0],
        mag[1],
        mag[2],
        gyro[0],
        gyro[1],
        gyro[2],
        accel[0],
        accel[1],
        accel[2])

In [5]:
def update_gyro_config(bno,page=0,bandwidth_code=0B111,range_code=0B000,verbose=False):

  # - Switch to config mode
  bno._config_mode()

  # Select page 1 for gyro configuration
  bno._write_byte(BNO055.BNO055_PAGE_ID_ADDR, page)
  time.sleep(0.05)

  gyro_config_address = 0XA
  config_value = ((bandwidth_code << 3) | range_code) & 0X3F

  if verbose:
    prior_value = bno._read_byte(gyro_config_address)
    time.sleep(0.05)
    print 'setting gyro config: {0} - {1!s}'.format(config_value,get_binary_word_str(config_value))

  # Update gyro settings
  bno._write_byte(gyro_config_address,config_value)
  time.sleep(0.05)

  if verbose:
    confirmation = bno._read_byte(gyro_config_address)
    print 'confirmation - from: {0} to: {1}'.format(prior_value & 0X3F,confirmation & 0X3F)
    print 'page ID: {}'.format(bno._read_byte(BNO055.BNO055_PAGE_ID_ADDR))

  # Select page 0 since done with gyro configuration
  bno._write_byte(BNO055.BNO055_PAGE_ID_ADDR, 0)

  # - Back into operation mode
  bno._operation_mode()

  time.sleep(0.65)

In [8]:
def run_bno_test():

  NUM_MEASUREMENTS = 100

  start_time = time.time()

  # Create and configure the BNO sensor connection.
  # Raspberry Pi configuration with serial UART and RST connected to GPIO 18:
  
  baudrate = 115200
  bno = BNO055_2(serial_port='/dev/ttyAMA0', rst=18, baudrate=baudrate)
  #bno = BNO055.BNO055(serial_port='/dev/ttyAMA0', rst=18, )
  #bno = BNO055.BNO055(serial_port='/dev/ttyAMA0', rst=18, serial_timeout_sec=None)
  print 'baudrate: ',bno._serial._baudrate
  
  done_creating_bno_time = time.time()

  # -- Reset bno and set to desired mode --
  mode = BNO055.OPERATION_MODE_NDOF
  #mode = BNO055.OPERATION_MODE_AMG
  #mode = BNO055.OPERATION_MODE_GYRONLY
  if not bno.begin(mode=mode):
    raise RuntimeError('Failed to initialize BNO055!')
  print 'Done starting BNO'

  # 12 Hz Gyro BW
  #update_gyro_config(bno,page=1,bandwidth_code=0B101,range_code=0B000,verbose=True)

  print 'Linear Acceleration\tMagnetometer\t\tGyroscope\t\tAccerlation'
  lin_accel = (0,0,0)
  mag = (0,0,0)
  gyro = (0,0,0)
  accel = (0,0,0)
  
  done_starting_bno_time = time.time()

  for i in range(NUM_MEASUREMENTS):

    lin_accel = bno.read_linear_acceleration()
    #mag = bno.read_magnetometer()
    #gyro = bno.read_gyroscope()
    #accel = bno.read_accelerometer()

    print_measurements(lin_accel,mag,gyro,accel)

  done_with_measurements_time = time.time()

  print '\ntime to create bno: {0}\ntime to start bno: {1}\ntime per measurement ({2}): {3}'.format(
    done_creating_bno_time - start_time,
    done_starting_bno_time - done_creating_bno_time,
    NUM_MEASUREMENTS,
    (done_with_measurements_time - done_starting_bno_time)/NUM_MEASUREMENTS)

In [7]:
run_bno_test()

about to create Serial object
baudrate:  115200
Done starting BNO
Linear Acceleration	Magnetometer		Gyroscope		Accerlation

time to create bno: 0.72398519516
time to start bno: 0.751362800598
time per measurement (100): 0.00397608995438


In [79]:
%prun -s time run_bno_test()

about to create Serial object
baudrate:  115200
Done starting BNO
Linear Acceleration	Magnetometer		Gyroscope		Accerlation

time to create bno: 0.661224842072
time to start bno: 0.754898071289
time per measurement (100): 0.0047691488266
 

115200: 0.0126917657852

In [ ]:
%%cython --annotate

In [5]:
from Adafruit_BNO055 import BNO055
import time

NUM_MEASUREMENTS = 1000

start_time = time.time()

# Create and configure the BNO sensor connection.
# Raspberry Pi configuration with serial UART and RST connected to GPIO 18:
bno = BNO055.BNO055(serial_port='/dev/ttyAMA0', rst=18)
#bno = BNO055.BNO055(serial_port='/dev/ttyAMA0', rst=18, serial_timeout_sec=None)

done_creating_bno_time = time.time()

# -- Reset bno and set to desired mode --

mode = BNO055.OPERATION_MODE_NDOF
#mode = BNO055.OPERATION_MODE_AMG
#mode = BNO055.OPERATION_MODE_GYRONLY
if not bno.begin(mode=mode):
  raise RuntimeError('Failed to initialize BNO055!')
print 'Done starting BNO'

lin_accel = (0,0,0)
mag = (0,0,0)
gyro = (0,0,0)
accel = (0,0,0)

done_starting_bno_time = time.time()

for i in range(NUM_MEASUREMENTS):

    lin_accel = bno.read_linear_acceleration()
    #mag = bno.read_magnetometer()
    #gyro = bno.read_gyroscope()
    #accel = bno.read_accelerometer()

done_with_measurements_time = time.time()

print '\ntime to create bno: {0}\ntime to start bno: {1}\ntime per measurement ({2}): {3}'.format(
    done_creating_bno_time - start_time,
    done_starting_bno_time - done_creating_bno_time,
    NUM_MEASUREMENTS,
    (done_with_measurements_time - done_starting_bno_time)/NUM_MEASUREMENTS)

Done starting BNO

time to create bno: 0.659081220627
time to start bno: 0.764106988907
time per measurement (100): 0.00368741989136
